## PARTIE 2 : SAT

In [1]:
import numpy as np
import random
from itertools import combinations, chain
import subprocess

from generate_dataset import convert_dataset, generate_dataset

## Création d'un dataset

In [2]:
students_count = 5
subjects_count = 3 #les différentes matières (critères)
classes_count = 3 #Accepté ou refusé

S, y = generate_dataset(students_count, subjects_count, classes_count)


Les coeffs sont =  [0.15 0.6  0.25]

Les frontières sont =  [[ 3. 14.]
 [ 2. 14.]
 [ 7. 18.]]

lambda =  0.5147881036111792

Les notes des élèves :
[[10 14 19]
 [ 5  6  5]
 [ 6  1 15]
 [ 9  6 10]
 [ 9 13  0]]

Les classes des élèves :
[2, 1, 0, 1, 1]


In [3]:
X = convert_dataset(S, y, classes_count)
print(X)

[array([[ 6,  1, 15]]), array([[ 5,  6,  5],
       [ 9,  6, 10],
       [ 9, 13,  0]]), array([[10, 14, 19]])]


## Variables

* ### x(i, h, k)

On crée les variables binaires x(i, h, k) signifiant "la note k pour le critère i correspond à la classe k" (et qui valent donc 1 dans ce cas, 0 sinon)

In [4]:
x=[]
for student in range(students_count):
    for subject in range (subjects_count):
        for student_class in range(1, classes_count):
            x.append((subject, student_class, S[student][subject]))
            
print(x)

[(0, 1, 10), (0, 2, 10), (1, 1, 14), (1, 2, 14), (2, 1, 19), (2, 2, 19), (0, 1, 5), (0, 2, 5), (1, 1, 6), (1, 2, 6), (2, 1, 5), (2, 2, 5), (0, 1, 6), (0, 2, 6), (1, 1, 1), (1, 2, 1), (2, 1, 15), (2, 2, 15), (0, 1, 9), (0, 2, 9), (1, 1, 6), (1, 2, 6), (2, 1, 10), (2, 2, 10), (0, 1, 9), (0, 2, 9), (1, 1, 13), (1, 2, 13), (2, 1, 0), (2, 2, 0)]


* ### y(B) 
On crée les variables binaires y(B) signifiant "la  coalition B est suffisante" (et qui valent 1 dans ce cas, 0 sinon)

In [5]:
#Ensemble des critères
I = list(range(subjects_count))

yB = []
for n in range(0,len(I)+1):
    yB.append([i for i in combinations(I,n)])

yB=list(chain.from_iterable(yB))
print(yB)

[(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]


### Traduction des variables en dictionnaires pour le solveur SAT

In [6]:
x2i = {v : i+1 for i,v in enumerate(set(x))} #numérotation qui commence à 1
#i2x = {i : v for v,i in x2i.items()}

print("x to Integers")
print(x2i)

print("\nIntegers to x")
#print(i2x)

x to Integers
{(1, 1, 13): 1, (2, 1, 0): 2, (2, 2, 5): 3, (0, 1, 6): 4, (0, 1, 9): 5, (2, 1, 15): 6, (1, 2, 14): 7, (0, 2, 10): 8, (1, 1, 6): 9, (0, 1, 5): 10, (2, 1, 5): 11, (2, 2, 10): 12, (2, 2, 19): 13, (1, 2, 1): 14, (1, 2, 13): 15, (0, 2, 6): 16, (0, 2, 9): 17, (1, 1, 14): 18, (2, 2, 0): 19, (0, 1, 10): 20, (2, 1, 10): 21, (1, 2, 6): 22, (2, 2, 15): 23, (2, 1, 19): 24, (0, 2, 5): 25, (1, 1, 1): 26}

Integers to x


In [7]:
y2i = {v : i+len(x2i)+1 for i,v in enumerate(yB)} #numérotation qui commence à taille de x2i + 1
#i2y = {i : v for v,i in y2i.items()}

print("yB to Integers")
print(y2i)

print("\nIntegers to yB")
#print(i2y)

yB to Integers
{(): 27, (0,): 28, (1,): 29, (2,): 30, (0, 1): 31, (0, 2): 32, (1, 2): 33, (0, 1, 2): 34}

Integers to yB


In [8]:
#Fusion des 2 dictionnaires
variables = {**x2i, **y2i}
print(variables)
i2variables = {i : v for v,i in variables.items()}
print(i2variables)

{(1, 1, 13): 1, (2, 1, 0): 2, (2, 2, 5): 3, (0, 1, 6): 4, (0, 1, 9): 5, (2, 1, 15): 6, (1, 2, 14): 7, (0, 2, 10): 8, (1, 1, 6): 9, (0, 1, 5): 10, (2, 1, 5): 11, (2, 2, 10): 12, (2, 2, 19): 13, (1, 2, 1): 14, (1, 2, 13): 15, (0, 2, 6): 16, (0, 2, 9): 17, (1, 1, 14): 18, (2, 2, 0): 19, (0, 1, 10): 20, (2, 1, 10): 21, (1, 2, 6): 22, (2, 2, 15): 23, (2, 1, 19): 24, (0, 2, 5): 25, (1, 1, 1): 26, (): 27, (0,): 28, (1,): 29, (2,): 30, (0, 1): 31, (0, 2): 32, (1, 2): 33, (0, 1, 2): 34}
{1: (1, 1, 13), 2: (2, 1, 0), 3: (2, 2, 5), 4: (0, 1, 6), 5: (0, 1, 9), 6: (2, 1, 15), 7: (1, 2, 14), 8: (0, 2, 10), 9: (1, 1, 6), 10: (0, 1, 5), 11: (2, 1, 5), 12: (2, 2, 10), 13: (2, 2, 19), 14: (1, 2, 1), 15: (1, 2, 13), 16: (0, 2, 6), 17: (0, 2, 9), 18: (1, 1, 14), 19: (2, 2, 0), 20: (0, 1, 10), 21: (2, 1, 10), 22: (1, 2, 6), 23: (2, 2, 15), 24: (2, 1, 19), 25: (0, 2, 5), 26: (1, 1, 1), 27: (), 28: (0,), 29: (1,), 30: (2,), 31: (0, 1), 32: (0, 2), 33: (1, 2), 34: (0, 1, 2)}


## Clauses

* ### Clause 1 : "échelle ascendente"

x(i,h,k') ∨ ¬x(i,h,k) (avec k<k')

Pour 2 notes k et k' sur un critère i, soit on a k' qui n'appartient pas à la classe h, soit k' appartient à h et dans ce cas k appartient aussi au moins à la classe h.

In [9]:
def clause1(classes_count, S, x, x2i):
    clauses_ascending_scales = []

    #On parcourt les matières
    for subject_index, subject in enumerate(S.T) : 
        #On parcourt les notes
        for student in subject:
            #On regarde les index des étudiants dont la note est inférieure
            students_inf_index = np.argwhere(subject < student)
            #Pour chaque classe
            for student_class in range (1, classes_count):
                variable_1 = x2i[(subject_index, student_class, student)]
                #Pour chaque étudiant dont la note est inférieure à l'étudiant regardé
                for index in students_inf_index:
                    variable_2 = x2i[(subject_index, student_class, subject[index[0]])]

                    if ([variable_1, -variable_2] not in clauses_ascending_scales):
                        clauses_ascending_scales.append([variable_1, -variable_2])

    return clauses_ascending_scales

clauses_ascending_scales=clause1(classes_count, S, x, x2i)
print(clauses_ascending_scales)

[[20, -10], [20, -4], [20, -5], [8, -25], [8, -16], [8, -17], [4, -10], [16, -25], [5, -10], [5, -4], [17, -25], [17, -16], [18, -9], [18, -26], [18, -1], [7, -22], [7, -14], [7, -15], [9, -26], [22, -14], [1, -9], [1, -26], [15, -22], [15, -14], [24, -11], [24, -6], [24, -21], [24, -2], [13, -3], [13, -23], [13, -12], [13, -19], [11, -2], [3, -19], [6, -11], [6, -21], [6, -2], [23, -3], [23, -12], [23, -19], [21, -11], [21, -2], [12, -3], [12, -19]]


* ### Clause 2 : "hiérarchie des profils"

x(i,h,k) ∨ ¬x(i,h',k) (avec 1<=h<h'<p-1)

Pour une note donnée k sur le critère i, soit la note appartient à la classe h, soit elle n'appartient pas à la classe h et dans ce cas elle n'appartient pas non plus à la classe du dessus.

**Remarque :** Dans le cas où on a que 2 classes (Accepté et Refusé par exemple), cet ensemble de clauses est vide car on a juste h = 1

In [10]:
def clause2(classes_count, S, x, x2i):
    clauses_hierarchy_profiles = []

    #On parcourt les classes
    for student_class in range(1, classes_count):
        #on regarde les classes qui sont inférieures :
        classes_inf = list(range(1, student_class))

        #on parcourt les matières:
        for subject_index, subject in enumerate(S.T) :
            #On parcourt les notes
            for student in subject :
                variable_1 = x2i[(subject_index, student_class, student)]
                #on parcourt les classes qui sont inférieures
                for classe_inf in classes_inf :
                    variable_2 = x2i[(subject_index, classe_inf, student)]

                    if ([variable_2, -variable_1] not in clauses_hierarchy_profiles):
                        clauses_hierarchy_profiles.append([variable_2, -variable_1])

    return(clauses_hierarchy_profiles)
     
clauses_hierarchy_profiles=clause2(classes_count, S, x, x2i)
print(clauses_hierarchy_profiles)

[[20, -8], [10, -25], [4, -16], [5, -17], [18, -7], [9, -22], [26, -14], [1, -15], [24, -13], [11, -3], [6, -23], [21, -12], [2, -19]]


* ### Clause 3 : "force de coalitions"

yB' ∨ ¬yB (avec B ⊂ B' ⊆ N )

Soit la coalition B n'est pas suffisante, soit elle l'est et dans ce cas B' l'est aussi.

In [11]:
def clause3(yB, y2i):
    clauses_coalition_strength = []

    #On parcourt les différentes coalitions
    for index, coalition in enumerate(yB):
        variable_1 = y2i[coalition]
        #On parcourt les coalitions plus petites
        coalitions_inf=[]
        for i in range(index):
            #si elle est inclue dans notre coalition
            if set(yB[i]).issubset(coalition):
                coalitions_inf.append(yB[i])

        for coalition_inf in coalitions_inf:
            variable_2 = y2i[coalition_inf]

            if ([variable_1, -variable_2] not in clauses_coalition_strength):
                clauses_coalition_strength.append([variable_1, -variable_2])
                
    return clauses_coalition_strength


clauses_coalition_strength=clause3(yB, y2i)
print(clauses_coalition_strength)

[[28, -27], [29, -27], [30, -27], [31, -27], [31, -28], [31, -29], [32, -27], [32, -28], [32, -30], [33, -27], [33, -29], [33, -30], [34, -27], [34, -28], [34, -29], [34, -30], [34, -31], [34, -32], [34, -33]]


* ### Clause 4 :

Vi∈B(¬x(i,h,ui)) ∨ ¬yB

Si toutes les notes de u (qui a été évalué par la classe h-1) sont au-dessus de la frontière de la classe h, alors B n'est pas une coalition suffisante.

Autre formulation : Soit la coalition B n'est pas suffisante, soit elle l'est et dans ce cas il y a au moins une note de u (évalué par la classe h-1) qui ne peut pas être de la classe h.

In [12]:
def clause4(classes_count, X, yB, y2i, x2i):
    clauses4= []

    #On parcourt les coalitions
    for coalition in yB:
        variable_1 = y2i[coalition]

        #On parcourt les classes
        for student_class in range(1, classes_count):
            
            #On parcourt les étudiants qui appartiennent à cette classe
            for student in X[student_class-1]:
                
                variables_list = [-variable_1]
                
                #On parcourt les matières
                for subject_index, subject in enumerate(student):
                    variable_2 = x2i[(subject_index, student_class, subject)]

                    variables_list.append(-variable_2)
        
                    if variables_list not in clauses4:
                        clauses4.append(variables_list)

    return clauses4

clauses4 = clause4(classes_count, X, yB, y2i, x2i)
print(clauses4)          

[[-27, -4, -26, -6], [-27, -25, -22, -3], [-27, -17, -22, -12], [-27, -17, -15, -19], [-28, -4, -26, -6], [-28, -25, -22, -3], [-28, -17, -22, -12], [-28, -17, -15, -19], [-29, -4, -26, -6], [-29, -25, -22, -3], [-29, -17, -22, -12], [-29, -17, -15, -19], [-30, -4, -26, -6], [-30, -25, -22, -3], [-30, -17, -22, -12], [-30, -17, -15, -19], [-31, -4, -26, -6], [-31, -25, -22, -3], [-31, -17, -22, -12], [-31, -17, -15, -19], [-32, -4, -26, -6], [-32, -25, -22, -3], [-32, -17, -22, -12], [-32, -17, -15, -19], [-33, -4, -26, -6], [-33, -25, -22, -3], [-33, -17, -22, -12], [-33, -17, -15, -19], [-34, -4, -26, -6], [-34, -25, -22, -3], [-34, -17, -22, -12], [-34, -17, -15, -19]]


* ### Clause 5 :

Vi∈B(x(i,h,ai)) ∨ yN\B

In [13]:
def clause5(classes_count, X, yB, y2i, x2i):
    clauses5= []

    #On parcourt les coalitions
    for coalition in yB:
        
        #Le complémentaire de cette coalition :
        coalition_comp = [elem for elem in yB if set(elem).isdisjoint(coalition) and set((*elem,*coalition)).issuperset(set(range(subjects_count)))][0]
        
        variable_1 = y2i[coalition_comp]

        #On parcourt les classes
        for student_class in range(1, classes_count):
            
            #On parcourt les étudiants qui appartiennent à cette classe
            for student in X[student_class]:
                
                variables_list = [variable_1]
                
                #On parcourt les matières
                for subject_index, subject in enumerate(student):
                    variable_2 = x2i[(subject_index, student_class, subject)]

                    variables_list.append(variable_2)
        
                    if variables_list not in clauses5:
                        clauses5.append(variables_list)

    return clauses5

clauses5 = clause5(classes_count, X, yB, y2i, x2i)
print(clauses5)          

[[34, 10, 9, 11], [34, 5, 9, 21], [34, 5, 1, 2], [34, 8, 7, 13], [33, 10, 9, 11], [33, 5, 9, 21], [33, 5, 1, 2], [33, 8, 7, 13], [32, 10, 9, 11], [32, 5, 9, 21], [32, 5, 1, 2], [32, 8, 7, 13], [31, 10, 9, 11], [31, 5, 9, 21], [31, 5, 1, 2], [31, 8, 7, 13], [30, 10, 9, 11], [30, 5, 9, 21], [30, 5, 1, 2], [30, 8, 7, 13], [29, 10, 9, 11], [29, 5, 9, 21], [29, 5, 1, 2], [29, 8, 7, 13], [28, 10, 9, 11], [28, 5, 9, 21], [28, 5, 1, 2], [28, 8, 7, 13], [27, 10, 9, 11], [27, 5, 9, 21], [27, 5, 1, 2], [27, 8, 7, 13]]


## Construction du DIMACS et résolution

In [14]:
def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./NCS/Exemple_SAT_Test/gophersat/win64/gophersat-1.1.6.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")
    
    return True, [int(x) for x in model[:-1] if int(x) != 0], { i2variables[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [15]:
#Lancer la résolution

myClauses= clauses_ascending_scales + clauses_hierarchy_profiles + clauses_coalition_strength + clauses4 + clauses5

myDimacs = clauses_to_dimacs(myClauses,len(variables))

write_dimacs_file(myDimacs,"workingfile.cnf")
res = exec_gophersat("workingfile.cnf")

#Résultat
print(res)

(True, [-1, -2, -3, 4, 5, -6, -7, 8, -9, 10, -11, -12, -13, -14, -15, -16, -17, -18, -19, 20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34], {(1, 1, 13): False, (2, 1, 0): False, (2, 2, 5): False, (0, 1, 6): True, (0, 1, 9): True, (2, 1, 15): False, (1, 2, 14): False, (0, 2, 10): True, (1, 1, 6): False, (0, 1, 5): True, (2, 1, 5): False, (2, 2, 10): False, (2, 2, 19): False, (1, 2, 1): False, (1, 2, 13): False, (0, 2, 6): False, (0, 2, 9): False, (1, 1, 14): False, (2, 2, 0): False, (0, 1, 10): True, (2, 1, 10): False, (1, 2, 6): False, (2, 2, 15): False, (2, 1, 19): False, (0, 2, 5): False, (1, 1, 1): False, (): False, (0,): False, (1,): False, (2,): False, (0, 1): False, (0, 2): False, (1, 2): False, (0, 1, 2): False})


Commentaire : Nous n'avons pas obtenu de résultat convenable (aucune des coalitions n'est suffisante par exemple). 